# Example 35: Flink SQL

Declarative processing with SQL.

## 1. Environment Setup

In [ ]:
# Install Java 11
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# Install Apache Flink
!pip install -q apache-flink

In [ ]:
from pyflink.table import EnvironmentSettings, TableEnvironment

# Create Table Environment
env_settings = EnvironmentSettings.in_streaming_mode()
t_env = TableEnvironment.create(env_settings)

# Define Source Table (Datagen)
t_env.execute_sql("""
    CREATE TEMPORARY TABLE source_table (
        id INT,
        val INT
    ) WITH (
        'connector' = 'datagen',
        'rows-per-second' = '5',
        'fields.id.kind' = 'sequence',
        'fields.id.start' = '1',
        'fields.id.end' = '20',
        'fields.val.min' = '1',
        'fields.val.max' = '100'
    )
""")

# Define Sink Table (Print)
t_env.execute_sql("""
    CREATE TEMPORARY TABLE print_sink (
        id INT,
        val INT,
        doubled_val INT
    ) WITH (
        'connector' = 'print'
    )
""")

# Process & Insert
t_env.execute_sql("""
    INSERT INTO print_sink
    SELECT id, val, val * 2 FROM source_table
""").wait()